In [1]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
import re
import time
import string

FILE = "american-english.txt"
ALLOWABLE_CHARACTERS = set(string.ascii_letters)
ALLOWED_ATTEMPTS = 6
WORD_LENGTH = 5

In [2]:
wordlist = [word.strip() for word in open(FILE, "r").readlines()]

#creating a set of valid words according to the rules of Wordle
WORDS = {
    word.lower()
    for word in wordlist    
    if len(word) == WORD_LENGTH and set(word) < ALLOWABLE_CHARACTERS
    }
WORDS

{'urban',
 'gutsy',
 'heirs',
 'treks',
 'alarm',
 'charm',
 'gazes',
 'curls',
 'nelly',
 'flame',
 'lumpy',
 'fiats',
 'orlon',
 'solon',
 'mangy',
 'trims',
 'lorie',
 'lyell',
 'aegis',
 'spiel',
 'shawl',
 'fades',
 'meets',
 'elmer',
 'sorry',
 'thymi',
 'sucks',
 'irate',
 'shard',
 'allen',
 'tusks',
 'loyal',
 'unsay',
 'worse',
 'grams',
 'palms',
 'izaak',
 'macho',
 'savor',
 'teats',
 'finer',
 'issue',
 'blair',
 'mania',
 'behan',
 'crumb',
 'swirl',
 'reese',
 'snipe',
 'pleas',
 'egged',
 'floyd',
 'shunt',
 'aerie',
 'creak',
 'hypes',
 'profs',
 'bauer',
 'dwelt',
 'serer',
 'sepoy',
 'makes',
 'aches',
 'waist',
 'harem',
 'thumb',
 'loire',
 'spawn',
 'druid',
 'pecan',
 'grime',
 'cleft',
 'kites',
 'nexus',
 'chain',
 'wispy',
 'yummy',
 'bayer',
 'belay',
 'lands',
 'fonda',
 'thurs',
 'lucas',
 'arose',
 'shoon',
 'miner',
 'karen',
 'baled',
 'stems',
 'cavil',
 'yakut',
 'gibed',
 'oiled',
 'clipt',
 'drain',
 'bouts',
 'stats',
 'crave',
 'gills',
 'reedy',


In [3]:
from collections import Counter
from itertools import chain

#Creating a Counter Object that counts the number of occurences
#of each letter used across all valid Wordle words

LETTER_COUNTER = Counter(chain.from_iterable(WORDS))

LETTER_COUNTER


Counter({'u': 1040,
         'r': 1927,
         'b': 733,
         'a': 2691,
         'n': 1471,
         'g': 682,
         't': 1517,
         's': 2940,
         'y': 854,
         'h': 828,
         'e': 3090,
         'i': 1728,
         'k': 602,
         'l': 1680,
         'm': 861,
         'c': 979,
         'z': 158,
         'f': 502,
         'p': 861,
         'o': 1877,
         'w': 475,
         'd': 1203,
         'v': 336,
         'x': 117,
         'q': 48,
         'j': 130})

In [4]:
#finding the total occurances of all letters
total = len(WORDS)*WORD_LENGTH
LETTER_FREQUENCY = {character: value / total 
                    for character, value in LETTER_COUNTER.items()}

LETTER_FREQUENCY

{'u': 0.035458574838049775,
 'r': 0.06570064780088647,
 'b': 0.02499147630412547,
 'a': 0.0917490623934538,
 'n': 0.05015342652574156,
 'g': 0.023252642345721105,
 't': 0.051721786566655305,
 's': 0.10023866348448687,
 'y': 0.029116945107398567,
 'h': 0.028230480736447322,
 'e': 0.10535288100920559,
 'i': 0.05891578588475963,
 'k': 0.02052505966587112,
 'l': 0.057279236276849645,
 'm': 0.029355608591885442,
 'c': 0.033378793044664165,
 'z': 0.0053869757927037165,
 'f': 0.017115581316058642,
 'p': 0.029355608591885442,
 'o': 0.06399590862598023,
 'w': 0.016195022161609274,
 'd': 0.04101602454824412,
 'v': 0.011455847255369928,
 'x': 0.0039890896692806,
 'q': 0.0016365496079099899,
 'j': 0.004432321854756222}

In [5]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += LETTER_FREQUENCY[char]
    return score / (WORD_LENGTH - len(set(word)) + 1)

In [6]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [7]:
def input_word():
    while True:
        word = input("Input the word you entered> ")
        if len(word) == WORD_LENGTH and word.lower() in WORDS:
            break
    return word.lower()

def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [8]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [9]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    time.sleep(1)
    finalEvaluation = list(word)
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [10]:
def solve():
    browser = webdriver.Firefox(executable_path = r'C:\Users\zbot6\WebDriver\geckodriver.exe')
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    yLetters = set()
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [11]:
solve()

Attempt 1 with 5866 possible words
arose      | 0.43 
raise      | 0.42 
aries      | 0.42 
arise      | 0.42 
earls      | 0.42 
reals      | 0.42 
laser      | 0.42 
aloes      | 0.42 
stare      | 0.41 
aster      | 0.41 
rates      | 0.41 
tares      | 0.41 
tears      | 0.41 
aisle      | 0.41 
elias      | 0.41 
QA--QAQA True
Attempt 2 with 207 possible words
stile      | 0.37 
stine      | 0.37 
sidle      | 0.36 
slide      | 0.36 
spire      | 0.36 
shire      | 0.36 
snide      | 0.36 
slice      | 0.36 
suite      | 0.35 
smile      | 0.35 
slime      | 0.35 
since      | 0.35 
spite      | 0.35 
smite      | 0.35 
snipe      | 0.34 
QA--QAQA True
Attempt 3 with 1 possible words
shire      | 0.36 
QA--QAQA True
Attempt 4 with 1 possible words
shire      | 0.36 
QA--QAQA True
Attempt 5 with 1 possible words
shire      | 0.36 
QA--QAQA True
Attempt 6 with 1 possible words
shire      | 0.36 
QA--QAQA True
